In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
# Kniznice

import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
from scipy import signal as sig
import random as rnd
import pandas as pd
import os
import glob, os
import scipy.signal as signal

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, adjusted_mutual_info_score
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics

# import keras

import cv2

# Priprava dat

In [ ]:
#Plotovanie
def plotovanie(time, signal):
    fig, axs = plt.subplots(figsize=(30, 10))
    plt.plot(time, signal)
    plt.show()


# Spectrak
def spektrak(signal):
    fig, axs = plt.subplots(figsize=(30, 10))
    axs.specgram(signal, NFFT=1024, noverlap=512)
    plt.show()

In [ ]:
def load_data(file_list):
    data = np.zeros((len(file_list), 200000))
    for i, file in enumerate(file_list):
        data[i] = np.loadtxt(file)
    return data

breakdown_pulse_cg_list = glob.glob("breakdown_pulse_cg/*.txt")
breakdown_pulse_cg = load_data(breakdown_pulse_cg_list)
print("breakdown_pulse_cg LOADED")

breakdown_pulse_ic_list = glob.glob("breakdown_pulse_ic/*.txt")
breakdown_pulse_ic = load_data(breakdown_pulse_ic_list)
print("breakdown_pulse_ic LOADED")

ic_activity_list = glob.glob("ic_activity/*.txt")
ic_activity = load_data(ic_activity_list)
print("ic_activity LOADED")

microsecond_pulse_list = glob.glob("microsecond_pulse/*.txt")
microsecond_pulse = load_data(microsecond_pulse_list)
print("microsecond_pulse LOADED")

narrow_bipolar_event_list = glob.glob("narrow_bipolar_event/*.txt")
narrow_bipolar_event = load_data(narrow_bipolar_event_list)
print("narrow_bipolar_event LOADED")

nothing_list = glob.glob("nothing/*__.txt")
nothing = load_data(nothing_list)
print("nothing LOADED")

return_stroke_list = glob.glob("return_stroke/*.txt")
return_stroke = load_data(return_stroke_list)
print("return_stroke LOADED")


# Priprava features

In [ ]:
data = {
    "signal":
    list(
        np.concatenate([
            breakdown_pulse_cg, breakdown_pulse_ic, ic_activity,
            microsecond_pulse, narrow_bipolar_event, nothing, return_stroke
        ])),
    "label":
    np.concatenate([
        np.array([1 for _ in range(len(breakdown_pulse_cg))]),
        np.array([2 for _ in range(len(breakdown_pulse_ic))]),
        np.array([3 for _ in range(len(ic_activity))]),
        np.array([4 for _ in range(len(microsecond_pulse))]),
        np.array([5 for _ in range(len(narrow_bipolar_event))]),
        np.array([6 for _ in range(len(nothing))]),
        np.array([7 for _ in range(len(return_stroke))])
    ])
}
data = pd.DataFrame(data)

In [ ]:
size = len(data)
data

In [ ]:
sos = signal.butter(5, 2000, 'hp', fs=200_000_000, output='sos', analog=False)
for i in range(size):
    data["signal"].at[i] = (np.cumsum(signal.sosfilt(sos, data["signal"][i])) / 2000)

In [ ]:
len(data["signal"][0])

In [ ]:
temp = np.zeros(size)

for i in range(size):
    temp[i] = data["signal"][i].max()

data["max"] = temp

In [ ]:
temp = np.zeros(size)

for i in range(size):
    temp[i] = data["signal"][i].std()

data["std"] = temp

In [ ]:
temp = np.zeros(size)

for i in range(size):
    temp[i] = data["signal"][i].sum()

data["sum"] = temp

In [ ]:
freq = 512
lines = 32

temp = np.zeros(size)
for j in range(0, int((freq/lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(data["signal"][i], nperseg=1024, noverlap=0, mode="psd")[2][1:513][0+j*lines:lines+j*lines].sum(axis=0).std()
    data["std__"+str(j)] = temp
    
temp = np.zeros(size)
for j in range(0, int((freq/lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(data["signal"][i], nperseg=1024, noverlap=0, mode="psd")[2][1:513][0+j*lines:lines+j*lines].sum(axis=0).max()
    data["max__"+str(j)] = temp
    
temp = np.zeros(size)
for j in range(0, int((freq/lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(data["signal"][i], nperseg=1024, noverlap=0, mode="psd")[2][1:513][0+j*lines:lines+j*lines].sum(axis=0).sum()
    data["sum__"+str(j)] = np.log(temp)

In [ ]:
data

# ML

In [ ]:
# Skalovanie

features = list(data.columns)
features.pop(0)
features.pop(0)
features

mm_scaler = MinMaxScaler()
mm_scaler.fit_transform(data[features])
data[features] = mm_scaler.transform(data[features])

In [ ]:
data

In [ ]:
features = list(data.columns)
features.pop(0)
features.pop(0)
features

X = data[features]
Y = data["label"]

In [ ]:
mm_scaler = MinMaxScaler()
mm_scaler.fit_transform(X)
mm_scaler.transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    shuffle=True,
                                                    train_size=0.8)

R2 = []
RMSE = []
AMI = []

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

predictions = linear_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
neighbors_model = KNeighborsRegressor(n_neighbors=3, weights="distance", p=1)
neighbors_model.fit(X_train, Y_train)

predictions = neighbors_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
strom_model = DecisionTreeClassifier(criterion="gini",
                                     max_depth=4,
                                     min_samples_split=2)
strom_model.fit(X_train, Y_train)

predictions = strom_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
les_model = RandomForestClassifier(n_estimators=100,
                                   criterion="gini",
                                   min_samples_split=2)
les_model.fit(X_train, Y_train)

predictions = les_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
rndtree_model = ExtraTreesClassifier(n_estimators=100,
                                     max_depth=None,
                                     min_samples_split=2,
                                     random_state=0)
rndtree_model.fit(X_train, Y_train)

predictions = rndtree_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
svc_model = SVC(C=1, degree=2)
svc_model.fit(X_train, Y_train)

predictions = svc_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
GBC_model = GradientBoostingClassifier(n_estimators=100,
                                       learning_rate=1,
                                       max_depth=5,
                                       random_state=1)
GBC_model.fit(X_train, Y_train)

predictions = GBC_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
# BEST of
Best_model = GradientBoostingClassifier(n_estimators=290,
                                        learning_rate=1,
                                        max_depth=1,
                                        random_state=0,
                                        validation_fraction=0.1)
Best_model.fit(X_train, Y_train)

predictions = Best_model.predict(X_test)

r2 = r2_score(Y_test, predictions)
rmse = mean_squared_error(Y_test, predictions, squared=False)
ami = adjusted_mutual_info_score(Y_test, np.round(predictions))

R2.append(r2)
RMSE.append(rmse)
AMI.append(ami)

print("R2= ", r2)
print("RMSE= ", rmse)
print("AMI= ", ami)

In [ ]:
labels = ["Linear",
    "KNN", "Rozhodovací strom", "Random forest", "Boosted random forest",
    "SVM", "Gradient Boosting", "XGBoost"
]

x = np.arange(len(labels))
width = 0.4

R2 = np.round_(R2, decimals=2, out=None)
RMSE = np.round_(RMSE, decimals=2, out=None)
AMI = np.round_(AMI, decimals=2, out=None)

#
R2 = np.round_(R2, decimals=2, out=None)
RMSE = np.round_(RMSE, decimals=2, out=None)
AMI = np.round_(AMI, decimals=2, out=None)
#

fig, ax = plt.subplots(figsize=(20, 5))
rects1 = ax.bar(x - width / 1.5, R2, width / 1.5, label="R2")
rects2 = ax.bar(x, RMSE, width / 1.5, label="RMSE")
rects3 = ax.bar(x + width / 1.5, AMI, width / 1.5, label="AMI")

ax.set_ylabel("R2; RMSE; AMI")
# ax.set_title("Presnost a chyba")
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.set_ylim([0, 1.5])


def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate("{}".format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha="center",
                    va="bottom")


autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()

In [ ]:
arr1inds = R2.argsort()
R2_sorted = R2[arr1inds]
RMSE_sorted = RMSE[arr1inds]
AMI_sorted = AMI[arr1inds]
labels_sorted = [" ",
    " ",
    " ",
    " ",
    " ",
    " ",
    " ",
    " ",
]
for i, j in enumerate(arr1inds):
    labels_sorted[i] = labels[j]

In [ ]:
x = np.arange(len(labels))
width = 0.4

fig, ax = plt.subplots(figsize=(20, 5))
rects1 = ax.bar(x - width / 1.5, R2_sorted, width / 1.5, label="R2")
rects2 = ax.bar(x, RMSE_sorted, width / 1.5, label="RMSE")
rects3 = ax.bar(x + width / 1.5, AMI, width / 1.5, label="AMI")

ax.set_ylabel("R2, RMSE, AMI")
# ax.set_title("Presnost a chyba")
ax.set_xticks(x)
ax.set_xticklabels(labels_sorted)
ax.legend()

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)
ax.set_ylim([0, 1.5])
fig.tight_layout()

plt.show()

In [ ]:
np.round(predictions)

In [ ]:
confusion_matrix = metrics.confusion_matrix(Y_test, np.round(predictions))

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

In [ ]:
fig, axs = plt.subplots(figsize=(7, 7))

cm_display.plot(ax=axs)

plt.show()

# Predict

In [ ]:
test_list = glob.glob("test/*.txt")

Check = np.zeros((len(test_list), 200000))
for i in range(len(test_list)):
    Check[i] = np.loadtxt(test_list[i])

CHECK = {
    "signal": list(np.concatenate([Check])),
    "label": np.concatenate([np.array([0 for _ in range(len(Check))])])
}
CHECK = pd.DataFrame(CHECK)

size_check = len(CHECK)
sos = signal.butter(10, 20000, 'hp', fs=200_000_000, output='sos', analog=False)
for i in range(size_check):
    CHECK["signal"].at[i] = (np.cumsum(signal.sosfilt(sos, CHECK["signal"][i])) / 2000)

In [ ]:
size = len(CHECK)

temp = np.zeros(size)

for i in range(size):
    temp[i] = CHECK["signal"][i].max()

CHECK["max"] = temp

temp = np.zeros(size)

for i in range(size):
    temp[i] = CHECK["signal"][i].std()

CHECK["std"] = temp

temp = np.zeros(size)

for i in range(size):
    temp[i] = CHECK["signal"][i].sum()

CHECK["sum"] = temp

freq = 512
lines = 32

temp = np.zeros(size)
for j in range(0, int((freq / lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(
            CHECK["signal"][i], nperseg=1024, noverlap=0,
            mode="psd")[2][1:513][0 + j * lines:lines +
                                  j * lines].sum(axis=0).std()
    CHECK["std__" + str(j)] = temp

temp = np.zeros(size)
for j in range(0, int((freq / lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(
            CHECK["signal"][i], nperseg=1024, noverlap=0,
            mode="psd")[2][1:513][0 + j * lines:lines +
                                  j * lines].sum(axis=0).max()
    CHECK["max__" + str(j)] = temp

temp = np.zeros(size)
for j in range(0, int((freq / lines))):
    for i in range(size):
        temp[i] = sig.spectrogram(
            CHECK["signal"][i], nperseg=1024, noverlap=0,
            mode="psd")[2][1:513][0 + j * lines:lines +
                                  j * lines].sum(axis=0).sum()
    CHECK["sum__" + str(j)] = np.log(temp)

In [ ]:
mm_scaler = MinMaxScaler()

mm_scaler.fit_transform(CHECK[features])
CHECK[features] = mm_scaler.transform(CHECK[features])

In [ ]:
file_object = open('predicted_list.txt', 'w')
file_object.close()

for line in range(len(CHECK)):
    ori_name = test_list[line]
    value = neighbors_model.predict(
        np.array(CHECK.iloc[line][1:-1]).reshape(1, -1))
    value = int(round(value[0]))
    if value == 1: name = "breakdown_pulse_cg"
    elif value == 2: name = "breakdown_pulse_ic"
    elif value == 3: name = "ic_activity"
    elif value == 4: name = "microsecond_pulse"
    elif value == 5: name = "narrow_bipolar_event"
    elif value == 6: name = "nothing"
    elif value == 7: name = "return_stroke"
    else: name = "ERR"
    file_object = open('predicted_list.txt', 'a')
    file_object.write(ori_name.split("\\")[1] + f'\t {name} \n')
    file_object.close()